In [1]:
from telegram.ext import Updater, CommandHandler, CallbackQueryHandler
from telegram.ext import MessageHandler, Filters
import collections
import sys
import time
import teleport
from telegram import KeyboardButton, ReplyKeyboardMarkup, Location
from bs4 import BeautifulSoup
import requests
import collections
import pandas as pd
from vincenty import vincenty
import math
import requests
import json
from pandas.io.json import json_normalize
pd.options.display.max_colwidth = 100
import matplotlib.pyplot as plt


In [2]:
def getting_data(long, lat):
    params = {'format': 'json'}
    g = requests.get('https://apidata.mos.ru/v1/features/629?api_key=65b653b36cd09a1fb0d6fe1223843c8a', params=params)
    data = json.loads(g.text)
    argh=json_normalize(data['features'])
    colnames=[]
    for i in argh.columns:
        colnames.append(i.split('.')[-1])
    argh.columns=colnames
    valid=argh.iloc[:,0:-5]
    givenpoint=[]
    listremover = lambda x: tuple(x[0])
    valid['coordinates']=valid['coordinates'].apply(listremover)
    valid['Distance']=valid['coordinates']
    givenpoint.append(long)
    givenpoint.append(lat)
    givenpoint=tuple(givenpoint)
    vince = lambda x: vincenty(x, givenpoint, miles=False)
    valid['Distance']=valid['Distance'].apply(vince)
    valid[valid['Distance'].idxmin():valid['Distance'].idxmin()+1]
    plt.figure()
    valid['Distance'].plot(kind='density')
    return(valid[valid['Distance'].idxmin():valid['Distance'].idxmin()+1])


In [3]:
def start(bot, update):
    
    keyboard = [[KeyboardButton("Нажимать сюда. P.S. Я просто делаю проект и не буду красть вашу дату;)", request_location=True)]]

    reply_markup = ReplyKeyboardMarkup(keyboard)

    update.message.reply_text("Привет, я бот, который может помочь найти ближайший к вам спортивный объект в Москве! Если вы расшарите свое местоположение, то я найду близжайшее спортивное учереждение к вам (・_・ヾ) Пожалуйста, расшарьте вашу локацию \^_-/", reply_markup=reply_markup)

coordinates=[]

def handle_location(bot, update):
    long=update.message.location.longitude
    lat=update.message.location.latitude
    global coordinates
    coordinates.append(long)
    coordinates.append(lat)
    print('aaaaa', long, lat)
    while coordinates==[]:
        sleep(10)
    update.message.reply_text("Вот, что я нашел совсем близко")
    update.message.reply_text("(ง ื▿ ื)ว")
    update.message.reply_text(str(getting_data(long, lat)['CommonName']).split('    ')[1].split('\nName')[0])
    update.message.reply_text("ФИО главы: "+str(getting_data(long, lat)['ChiefName']).split('    ')[1].split('\nName')[0])
    contacts=["WorkingHours",'ChiefPhone','PublicPhone','Email']
    


    for i in contacts:
        update.message.reply_text(str(getting_data(long, lat)[i]).split('    ')[1].split('\nName')[0].split('{')[1].split('}')[0].replace("'", '')   )
    place = [list(x) for x in getting_data(long, lat)['coordinates']]
    target_lat=place[0][1]
    target_long=place[0][0]
    bot.sendLocation(chat_id=update.message.chat_id, latitude=target_lat, longitude=target_long)
    update.message.reply_text("Спасибо за пользование! Я также могу показать результаты поиска для нового нажатия на кнопку;)")
    return(getting_data(long, lat))



    


In [4]:
updater = Updater(token='559576587:AAFggNF96clfCiBwwB0GkwJTmudscMQqoCY')  # тут токен, который выдал вам Ботский Отец!

start_handler = CommandHandler('start', start)  

location_handler = MessageHandler(Filters.location, handle_location)

updater.dispatcher.add_handler(start_handler) 
updater.dispatcher.add_handler(location_handler) 

updater.start_polling()  # поехали!